In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# train.head()

In [4]:
# Do preprocessing :)
# test.info()
# train.info()

In [5]:
# train.describe()

In [6]:
train['Donation Rate'] = train['Frequency (times)']/(train['Time (months)'] + 1)
test['Donation Rate'] = test['Frequency (times)']/(test['Time (months)']  +1 )

In [7]:
train['Donation Distance'] = (train['Time (months)']/(train['Frequency (times)'] +1))
test['Donation Distance'] = (test['Time (months)']/(test['Frequency (times)'] +1))

In [8]:
train['is rest time'] = train['Recency (months)']<=3
test['is rest time'] = test['Recency (months)']<=3

In [9]:
def calcRepeatedYears(near,far):
    if near%12==0:
        return (far//12) - (near//12) +1
    return (far//12) - (near//12)
train['Repeated Years'] = train.apply(lambda r : calcRepeatedYears(r['Recency (months)'],r['Time (months)']),axis=1)
test['Repeated Years'] = test.apply(lambda r : calcRepeatedYears(r['Recency (months)'],r['Time (months)']),axis=1)

In [10]:
train['Frequency per Year'] = train['Frequency (times)']/(train['Time (months)']/12 +1)
test['Frequency per Year'] = test['Frequency (times)']/(test['Time (months)']/12 +1)

In [11]:
train['Recency Rate'] = train['Recency (months)']/(train['Time (months)'] + 1)
test['Recency Rate'] = test['Recency (months)']/(test['Time (months)']  +1 )

In [12]:
# train.corr()

In [13]:
train = train.drop('Monetary (c.c. blood)',axis=1)
test = test.drop('Monetary (c.c. blood)',axis=1)

In [14]:
target = 'whether he/she donated blood in March 2007'

In [15]:
np.abs(train.corr()[target]).sort_values()

Time (months)                                 0.028481
Repeated Years                                0.041729
is rest time                                  0.182998
Recency Rate                                  0.214805
Frequency (times)                             0.218661
Donation Distance                             0.241318
Recency (months)                              0.274505
Donation Rate                                 0.314568
Frequency per Year                            0.344419
whether he/she donated blood in March 2007    1.000000
Name: whether he/she donated blood in March 2007, dtype: float64

In [16]:
# train[train['Donation Rate']>1]

In [17]:
# train.describe()

In [18]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# cols = train.columns
# n_cols = 4 
# n_rows = int(np.ceil(len(cols) / n_cols))

# plt.figure(figsize=(4 * n_cols, 5 * n_rows))

# for i, col in enumerate(cols, 1):
#     plt.subplot(n_rows, n_cols, i)
#     sns.histplot(train[col],kde=True)
#     plt.title(col)
#     plt.grid(True)

# plt.tight_layout()
# plt.show()

In [19]:
# train.columns

In [20]:
# train[train.columns[4]].value_counts()

In [21]:
x = train.drop(target,axis=1)
y = train[target]

In [22]:
x.iloc[0]

Recency (months)             16
Frequency (times)             2
Time (months)                70
Donation Rate          0.028169
Donation Distance     23.333333
is rest time              False
Repeated Years                4
Frequency per Year     0.292683
Recency Rate           0.225352
Name: 0, dtype: object

In [23]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, include_bias=False)

x_poly = poly.fit_transform(x)
test_poly  = poly.transform(test)

feature_names = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=feature_names, index=x.index)
test  = pd.DataFrame(test_poly, columns=feature_names, index=test.index)

In [24]:
x.head()

,Recency (months),Frequency (times),Time (months),Donation Rate,Donation Distance,is rest time,Repeated Years,Frequency per Year,Recency Rate,Recency (months)^2,...,Repeated Years^3,Repeated Years^2 Frequency per Year,Repeated Years^2 Recency Rate,Repeated Years Frequency per Year^2,Repeated Years Frequency per Year Recency Rate,Repeated Years Recency Rate^2,Frequency per Year^3,Frequency per Year^2 Recency Rate,Frequency per Year Recency Rate^2,Recency Rate^3
0,16.0,2.0,70.0,0.028169,23.333333,0.0,4.0,0.292683,0.225352,256.0,...,64.0,4.682927,3.605634,0.342653,0.263827,0.203134,0.025072,0.019304,0.014863,0.011444
1,11.0,10.0,35.0,0.277778,3.181818,0.0,2.0,2.553191,0.305556,121.0,...,8.0,10.212766,1.222222,13.037574,1.560284,0.186728,16.643711,1.991852,0.238377,0.028528
2,13.0,4.0,21.0,0.181818,4.200000,0.0,0.0,1.454545,0.590909,169.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.077385,1.250188,0.507889,0.206330
3,11.0,4.0,27.0,0.142857,5.400000,0.0,2.0,1.230769,0.392857,121.0,...,8.0,4.923077,1.571429,3.029586,0.967033,0.308673,1.864360,0.595097,0.189953,0.060632
4,4.0,5.0,23.0,0.208333,3.833333,0.0,1.0,1.714286,0.166667,16.0,...,1.0,1.714286,0.166667,2.938776,0.285714,0.027778,5.037901,0.489796,0.047619,0.004630


In [25]:
# from sklearn.preprocessing import StandardScaler

# stds = StandardScaler()
# for col in test.columns:
#     x[col] = stds.fit_transform(x[[col]])
#     test[col] = stds.transform(test[[col]])
# train.describe()

In [26]:
# train['is rest time'].value_counts()

In [27]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
for col in test.columns:
    x[col] = mms.fit_transform(x[[col]])
    test[col] = mms.transform(test[[col]])
# train.describe()

In [28]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# x = train.drop(target,axis=1)
# y = train[target]

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=.2,random_state=22)

sampler = SMOTE()
x_train,y_train = sampler.fit_resample(x_train,y_train)

In [29]:
y_train.value_counts()

whether he/she donated blood in March 2007
0    409
1    409
Name: count, dtype: int64

In [30]:
 # LogisticRegression?

In [ ]:
# Model Target column :)
from sklearn.model_selection import GridSearchCV


model = LogisticRegression(max_iter=10000)
params = {
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100 , 1000]
}
grid_search = GridSearchCV(model, param_grid=params,cv=5)
grid_search.fit(x_train, y_train)
model = grid_search.best_estimator_
y_pred = model.predict(x_test)

In [ ]:
# model = LogisticRegression(max_iter=10000,n_jobs=-1,penalty='l2',solver='lbfgs',C=1)
# model.fit(x_train,y_train)
# y_pred = model.predict(x_test)

In [ ]:
# Evaluete model :)
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='macro'))

In [ ]:
from sklearn.model_selection import cross_validate, KFold
k_fold = KFold(5)
cross_validate(model, x_train, y_train, cv = k_fold, return_train_score=True)

In [37]:
# max_iter = 10
# f1_scores = []
# for _ in range(max_iter):
#     xtr,xte,ytr,yte = train_test_split(x,y, test_size=.2)
#     m = LogisticRegression()
#     grid_search = GridSearchCV(m, param_grid=params, cv=5)
#     grid_search.fit(xtr, ytr)
#     m = grid_search.best_estimator_
#     yp = model.predict(xte)
#     f1_scores.append(f1_score(yte, yp, average='macro'))

In [38]:
# np.mean(f1_scores)

In [37]:
sampler = SMOTE()
x,y = sampler.fit_resample(x,y)
# model = LogisticRegression(max_iter=10000,n_jobs=-1)
model.fit(x,y)
from sklearn.model_selection import cross_validate, KFold
k_fold = KFold(5)
cross_validate(model, x, y, cv = k_fold, return_train_score=True)

{'fit_time': array([1.03128433, 0.98152018, 0.97884846, 0.96835136, 1.06369114]),
 'score_time': array([0.0057838 , 0.00573301, 0.00613236, 0.00399971, 0.0050106 ]),
 'test_score': array([0.61165049, 0.65853659, 0.62439024, 0.63414634, 0.51707317]),
 'train_score': array([0.71097561, 0.69427527, 0.70645554, 0.72959805, 0.73081608])}

In [ ]:
# Predict for test data :)
submission = pd.DataFrame({'prediction':model.predict(test)})
submission

In [ ]:
submission.value_counts()

In [ ]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)


joblib.dump(model, 'model')
submission.to_csv('submission.csv', index=False)

file_names = ['model', 'submission.csv', 'blood_donation.ipynb']
compress(file_names)